In [4]:
from pyprojroot import here
import duckdb

In [8]:

raw_folder = here() / "src" / "raw"
src_folder = here() / "src"
query_folder =  here() / "src" / "query"
processed_folder = here() / "src" / "processed"
scale_factor = 1

print(raw_folder)

c:\Users\JonasHertz\git\bispecialist\raspberry_duckdb_tcph_test\src\raw


In [14]:
# Create duckdb database
con = duckdb.connect()

In [16]:

# Generate TPC-H data
con.execute("INSTALL tpch; LOAD tpch;")
con.execute(f"CALL dbgen(sf={scale_factor});")  # sf = scale factor

In [17]:
tables = ["nation", "region", "part", "supplier", "partsupp", "customer", "orders", "lineitem"]

(raw_folder / f"{scale_factor:02d}").mkdir(parents=True, exist_ok=True)

for table in tables:
    path = raw_folder / f"{scale_factor:02d}" / f"{table}.parquet"
    con.execute(f"COPY {table} TO '{path}' (FORMAT 'parquet');")

con.close()

In [18]:
con = duckdb.connect()
queries = con.execute("SELECT * FROM tpch_queries()").fetchall()

for i, row in enumerate(queries, 1):
    sql = row[1]
    (query_folder / f"tpch_q{i:02}.sql").write_text(sql)

con.close()